# Sentence Classification

- 감성 분석(Sentiment Classification), 문서 쌍 분류(NLI)를 하기 위한 Bert Classification Model을 만듬


- 해당 모델에서는 데이터을 넣으면 감성 분석을 해야하는 경우에는 감성 분석을 문서 쌍 분류를 해야하는 경우에는 문서 쌍 분류를 하는 모델임


- 해당 모델을 사용하기 위해 전처리 함수를 만들어 놓았으나 데이터에 어느 정도의 전처리가 필요
  - 변수명의 통일이 필요함 document_1, document_2(문서 쌍 분류의 경우), label

- 전처리 함수 들어가면 중복 데이터, 결측치, id컬럼을 제거함

- label의 데이터 타입이 str인 경우 str_label 컬럼을 만들어 줌


In [ ]:
!pip3 install transformers

     |████████████████████████████████| 3.8 MB 4.3 MB/s 
     |████████████████████████████████| 6.5 MB 50.3 MB/s 
     |████████████████████████████████| 67 kB 5.0 MB/s 
     |████████████████████████████████| 895 kB 49.8 MB/s 
     |████████████████████████████████| 596 kB 48.5 MB/s 
  Attempting uninstall: pyyaml
    Found existing installation: PyYAML 3.13
    Uninstalling PyYAML-3.13:
      Successfully uninstalled PyYAML-3.13


In [ ]:
import os
import json
import urllib.request

import pandas as pd
import numpy as np
import random
import time
import datetime

import torch
from torch.optim import Adam
from torch.utils.data import DataLoader,TensorDataset,RandomSampler
from transformers import BertTokenizer,BertTokenizerFast
from transformers import get_linear_schedule_with_warmup
from transformers import BertConfig, BertForSequenceClassification, AdamW

from transformers import TextClassificationPipeline

from tqdm import tqdm
import gc

from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score 
from sklearn.preprocessing import LabelEncoder

from google.colab import drive
drive.mount('/content/gdrive', force_remount=True)

Mounted at /content/gdrive


In [ ]:
%cd /content/gdrive/My Drive/NLP

/content/gdrive/My Drive/NLP


In [ ]:
# 재현을 위해 랜덤시드 고정
seed_val = 42
random.seed(seed_val)
np.random.seed(seed_val)
torch.manual_seed(seed_val)
torch.cuda.manual_seed_all(seed_val)

#  1. 데이터 전처리


In [ ]:
def drop_na_and_duplicates(df):
    df = df.dropna()
    df = df.drop_duplicates()
    df = df.reset_index(drop=True)
    df = df.drop('id',axis=1) if 'id' in df.columns else df
    try:
        df.columns = ['document_1','document_2','str_label'] if df.iloc[:,-1].dtypes == 'O' else ['document_1','document_2','label']
    except Exception:
        df.columns = ['document_1','str_label'] if df.iloc[:,-1].dtypes == 'O' else ['document_1','label']

    df['label'] = LabelEncoder().fit_transform(df['str_label'].tolist()) if 'str_label' in df.columns else df['label']

    return df

## 모델 불러오기 및 데이터 전처리 함수를 통해 전처리

### DATA 1. 네이버 영화 리뷰

- **긍정, 부정에 대한 감정 분석**

In [ ]:
!git clone https://github.com/e9t/nsmc.git

train_data = pd.read_csv("nsmc/ratings_train.txt", sep='\t')
train_data = drop_na_and_duplicates(train_data)

map_dict = {0 : '부정', 1 : '긍정'}

dev_data = pd.read_csv("nsmc/ratings_test.txt", sep='\t')
dev_data = drop_na_and_duplicates(dev_data)

fatal: destination path 'nsmc' already exists and is not an empty directory.


In [ ]:
print(f'train data 개수 : {len(train_data)} \ndev data 개수 : {len(dev_data)}')
print(map_dict)
display(train_data.sample(5))
display(dev_data.sample(5))

train data 개수 : 149995 
dev data 개수 : 49997
{0: '부정', 1: '긍정'}


,document_1,label
128072,원본이 최고,1
65515,스릴감과 훈훈함이 있는 영화.,1
85046,굉장히 저평가되는 영화중 하나라고 생각함,1
118189,정말영화같은이야기 영화여서 영화같은이야기가 좋다,1
92366,계기도없는데 이상하다,0


,document_1,label
27461,꼭 보세요. 많은 생각이 듭니다.,1
21688,새벽에 봤는데 잠 안올 정도로 재밌었네요,1
41303,옥택연이연희 오그라드는 연기하며.. (어느정도냐면 이희준이 봐줄만한 수준) 제작진은...,0
21363,히치콕은역시천재최근스릴러들보다수준이높다,1
3630,이런 싱거운 드라큐라를 보았는가...음향소리 때문에 귀가 찢어질 듯,0


### DATA 2. 챗봇 데이터 

- **공포, 놀람, 중립, 분노, 슬픔, 행복, 혐오 ,공포 총 7개의 감정에 대한 감정 분석**

- 한국어 감정 정보가 포함된 단발성 대화 데이터셋

In [ ]:
chatbot_data = pd.read_excel('datas/korean_chatbot.xlsx')
chatbot_data = drop_na_and_duplicates(chatbot_data)

map_dict = dict(zip(chatbot_data['str_label'].unique(),chatbot_data['label'].unique()))

train_data, dev_data, train_label, dev_label = train_test_split(chatbot_data[['document_1','str_label']],chatbot_data['label'], test_size=0.2, random_state=42, shuffle=True)

train_data['label'] = train_label
dev_data['label'] = dev_label

In [ ]:
print(f'train data 개수 : {len(train_data)} \ndev data 개수 : {len(dev_data)}')
print(map_dict)
display(train_data.sample(5))
display(dev_data.sample(5))

train data 개수 : 30822 
dev data 개수 : 7706
{'공포': 0, '놀람': 1, '분노': 2, '슬픔': 3, '중립': 4, '행복': 5, '혐오': 6}


,document_1,str_label,label
7275,이런적 있으신분들 계신가요?,놀람,1
24223,상술이 넘치는구나...,중립,4
2805,나 이제 남은3주 겁난다.,공포,0
37269,꼬식이에 곰팡이 핀 새끼들 뇌까지 곰팡이폈냐,혐오,6
4809,저가받는 불이익은 뭐가잇을까요???,공포,0


,document_1,str_label,label
21956,하 진짜 다시 재회하면 좋을줄 아랐는데 아니네요....,슬픔,3
35892,우리나라가 언제부터 원터미팅까지 신경쓰게됐지?ㅋㅋ,혐오,6
28025,나는 이래서 존스가 좋다,행복,5
1484,일본놈들 어떻게 살냐 저런데서 개무섭겠다ㄷㄷ,공포,0
27548,"v오빤 언제나 저에겐 최고에요 , 오홍홍",행복,5


### DATA 3 카카오 브레인 데이터

- **document_1과 document_2의 문장 관계를 파악**
 - 문서 쌍 분류로 NLI라고 불림

- 간혹 BertForNextSentence 사용한다는 글이 있으니 나중에 꼭 해보자!

In [ ]:
# 훈련 데이터 다운로드
urllib.request.urlretrieve("https://raw.githubusercontent.com/kakaobrain/KorNLUDatasets/master/KorNLI/multinli.train.ko.tsv", filename="multinli.train.ko.tsv")
urllib.request.urlretrieve("https://raw.githubusercontent.com/kakaobrain/KorNLUDatasets/master/KorNLI/snli_1.0_train.ko.tsv", filename="snli_1.0_train.ko.tsv")

# 검증 데이터 다운로드
urllib.request.urlretrieve("https://raw.githubusercontent.com/kakaobrain/KorNLUDatasets/master/KorNLI/xnli.dev.ko.tsv", filename="xnli.dev.ko.tsv")

# 테스트 데이터 다운로드
urllib.request.urlretrieve("https://raw.githubusercontent.com/kakaobrain/KorNLUDatasets/master/KorNLI/xnli.test.ko.tsv", filename="xnli.test.ko.tsv")

train_snli = pd.read_csv("snli_1.0_train.ko.tsv", sep='\t', quoting=3)
train_xnli = pd.read_csv("multinli.train.ko.tsv", sep='\t', quoting=3)
train_data = train_snli.append(train_xnli)

dev_data = pd.read_csv("xnli.dev.ko.tsv", sep='\t', quoting=3)
test_data = pd.read_csv("xnli.test.ko.tsv", sep='\t', quoting=3)

train_data = drop_na_and_duplicates(train_data)
dev_data = drop_na_and_duplicates(dev_data.append(test_data))

map_dict = dict(zip(train_data['label'].unique(), train_data['str_label'].unique()))

In [ ]:
print(f'train data 개수 : {len(train_data)} \ndev data 개수 : {len(dev_data)}')
print(map_dict)
display(train_data.sample(5))
display(dev_data.sample(5))

train data 개수 : 941814 
dev data 개수 : 7500
{2: 'neutral', 0: 'contradiction', 1: 'entailment'}


,document_1,document_2,str_label,label
134067,"이 사진에는 세 사람이 있는데, 어떤 물체를 훑어보고 있는 한 남자, 사진에 없는 ...",입을 크게 벌린 소녀가 사진 속에 있다.,entailment,1
204226,야구팀 투수가 타자에게 공을 던진다.,야구 투수가 타자에게 스트라이크를 날렸다.,neutral,2
296873,야외 시장에서 두 남자.,남자들이 밖에 있다.,entailment,1
940226,"1993년부터 1994년까지 싱글피스 퍼스트클래스 볼륨은 0.2% 감소했지만, 사전...",1등석 단품은 0.2% 하락했다.,entailment,1
146886,그 배는 두 개의 큰 돛대를 가지고 있다.,타조는 땅에 머리를 박고 있다.,contradiction,0


,document_1,document_2,str_label,label
1998,남부의 농담에는 계급의 연관성을 뒤집은 한 가지 세부 사항이 있다.,"japes의 남부에 여름에는 5천 명, 겨울에는 2천 명의 인구가 있다.",neutral,2
1615,미국에서 모금된 일부 자금이 알카에다 또는 그 계열 조직으로 옮겨졌을지라도 미국은 ...,미국은 확실히 알카에다에게 돈을 주지 않았다.,contradiction,0
599,디에고는 그녀의 지시를 따랐고 언덕 꼭대기에서 여전히 이슬로 뒤덮인 아름다운 카스티...,언덕 꼭대기에 장미가 있었다.,entailment,1
1674,그녀와의 관계는 미국에서 그의 시간 내내 가깝게 유지되었다.,그는 미국 여성과 관계를 가지고 있었다.,neutral,2
4276,"알겠습니다, 선생님, 아직 상황을 파악하지 못하셨군요.",그는 모든 사실을 모르기 때문에 상황을 이해하지 못했다.,neutral,2


# 2. 모델에 사용하기 위한 전처리

train data 전처리와 test data 전처리를 따로 분리해놓음

## 2-1. Data Preprocessing 

In [ ]:
def preprocess(train_data, batch_size=32, method = 'train'):
    if 'document_2' not in train_data.columns:
        sent_1 = [str(sentence) for sentence in  train_data['document_1'].tolist()]
        batch_input = tokenizer(sent_1, padding=True, truncation=True)
    
    # NLI의 경우
    elif 'document_2' in train_data.columns:
        sent_1 = [str(sentence) for sentence in  train_data['document_1'].tolist()]
        sent_2 = [str(sentence) for sentence in  train_data['document_2'].tolist()]
        batch_input = tokenizer(sent_1, sent_2, padding=True, truncation=True)


    batch_input = {key : torch.tensor(value) for key, value in batch_input.items()}

    batch_input['labels'] = torch.tensor(train_data['label'].values)
    
    
    if method == 'train' or method == 'valid': 
        batch_input['labels'] = torch.tensor(train_data['label'].values)
        dataset = TensorDataset(batch_input['input_ids'], batch_input['attention_mask'], batch_input['token_type_ids'], batch_input['labels'])
        if method == 'train':
            sampler = RandomSampler(dataset)
            dataloader = DataLoader(dataset, sampler=sampler, batch_size=batch_size)
        elif method == 'valid':
            dataloader = DataLoader(dataset, batch_size=batch_size)
            
    elif method == 'test':
        dataset = TensorDataset(batch_input['input_ids'], batch_input['attention_mask'], batch_input['token_type_ids'])
        dataloader = DataLoader(dataset, batch_size=batch_size)
        
    return dataloader

# 3. model 

## 1. model train

In [ ]:
 def train_one_epoch(optimizer, scheduler, dataloader):

     model.train()

     train_loss = 0.0

     for batchs in tqdm(dataloader):
         batch = tuple(b.to(device) for b in batchs)

         inputs = {
                 "input_ids": batch[0],
                 "attention_mask": batch[1],
                 "token_type_ids": batch[2],
                 'labels' : batch[3]
             }

         optimizer.zero_grad()

         output = model(**inputs)
         
         loss = output[0]
         
         loss.backward()

         torch.nn.utils.clip_grad_norm_(model.parameters(), 1.0)

         optimizer.step()
         scheduler.step()
        
         train_loss += loss.item()

     avg_train_loss = train_loss / len(dataloader)

     return avg_train_loss

## 2. model evaluation

In [ ]:
# 이거로 val_data와 test_data를 평가 가능
def evaluate_one_epoch(dataloader): 
    preds = []
    model.eval()

    eval_loss, eval_accuracy = 0, 0

    for batchs in tqdm(dataloader):
        batch = tuple(b.to(device) for b in batchs)
        inputs = {
                 "input_ids": batch[0],
                 "attention_mask": batch[1],
                 "token_type_ids": batch[2]
             }

        with torch.no_grad():
             output = model(**inputs)
                
        logits = output[0]

        # CPU로 데이터 이동
        logits = logits.detach().cpu()
        preds.append(logits)

    preds = torch.cat(preds, dim=0).argmax(dim=-1).tolist()

    return preds

## 3. model

In [ ]:
def sentiment_analysis_model(train_data, dev_data, lr=1e-4,epochs = 4, batch_size=32,bert='klue/bert-base', save=True, path='bert_tc'):
    gc.collect()
    torch.cuda.empty_cache()

    global tokenizer
    tokenizer = BertTokenizerFast.from_pretrained(bert)

    global model, device 
    model = BertForSequenceClassification.from_pretrained(bert,num_labels=len(train_data.iloc[:,-1].value_counts()))
    device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
    model = model.to(device)
    
    train_dataloader  = preprocess(train_data,batch_size=batch_size, method='train')
    valid_dataloader  = preprocess(dev_data, batch_size=batch_size, method='valid')
    print('')
    print('Data Preprocessing Complete!')


    optimizer = Adam(model.parameters(),
                    lr = lr, # 학습률
                    eps = 1e-8)

    total_steps = len(train_dataloader) * epochs

    scheduler = get_linear_schedule_with_warmup(optimizer, 
                                                num_warmup_steps = 0,
                                                num_training_steps = total_steps)

    print('')
    print('Model Training Start')
    print(f'Epochs : {epochs} / Learning Rate : {str(lr)} / Batch Size : {batch_size}')
    print('')

    loss = []
    acc = []
    for epoch in range(1,epochs+1):
        gc.collect()
        torch.cuda.empty_cache()
        print(f"epoch = {epoch}")

        train_loss = train_one_epoch(optimizer, scheduler, dataloader=train_dataloader)
        loss.append(train_loss)

        preds = evaluate_one_epoch(dataloader=valid_dataloader)

        val_acc = accuracy_score(dev_data['label'].tolist(),preds)
        acc.append(val_acc)
        
        print(f"Loss = {train_loss:.3f} / Accuracy = {val_acc:.3f}")
        print('')

        if save:
            model.save_pretrained(f'models/{path}')
            tokenizer.save_pretrained(f'models/{path}')
            print('Model Save')

    print('')
    print("Training Complete!")

    return {'loss':loss,'acc':acc}

In [ ]:
# bert = 'klue/bert-base'
# bert = 'beomi/kcbert-base'

score = sentiment_analysis_model(train_data, dev_data,lr=1e-5,batch_size=32 ,epochs = 4, bert = 'klue/bert-base', path='multi_class_test')

# 4. Test

In [ ]:
def Sentiment_Analysis(test_data, batch_size=32,bert='nsmc_test'):
    gc.collect()
    torch.cuda.empty_cache()

    global tokenizer
    tokenizer = BertTokenizer.from_pretrained(f'klue/bert-base')
    test_dataloader = preprocess(test_data, batch_size=batch_size, method='test')

    global model, device
    model = BertForSequenceClassification.from_pretrained(f'models/{bert}')
    device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
    model = model.to(device)

    pred_labels = evaluate_one_epoch(dataloader=test_dataloader)

    new_df = test_data[['document_1','document_2']].copy() if 'document_2' in train_data.columns else test_data[['document_1']].copy()
    new_df['pred_labels'] = pred_labels
    new_df['str_label'] = new_df['pred_labels'].apply(lambda x : map_dict[x])
    
    return  new_df

In [ ]:
df = Sentiment_Analysis(dev_data, batch_size=32,bert='nsmc_test')    
df.sample(5)

100%|██████████| 1563/1563 [03:17<00:00,  7.92it/s]


,document_1,pred_labels,str_label
46814,내 깅코를 ㅜ,1,긍정
48546,일본영화에다가 사극이라는 점을 제외해도 이정도면 잘 만든거다- _-,1,긍정
1324,시나리오 두번 다시 쓰지마라,0,부정
19874,생각보다 너무 슬프고 재밌다. 이 정도로 폐인 될 줄 몰랐음,1,긍정
26732,1회부터 계속 봐오던 통쾌한 복수를 예상했던 내가 바보,0,부정
